<a href="https://colab.research.google.com/github/ruthbrennankk/scalable_group_project/blob/master/captcha_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
!pip3 install --index-url https://google-coral.github.io/py-repo/ tflite_runtime
!pip install captcha
!pip install stsci.ndimage

In [ ]:
# Imports
import os
import cv2
import numpy as np
import string
import random
import argparse
import tensorflow as tf
import tensorflow.keras as keras
import scipy.ndimage
import random
import captcha.image

from PIL import Image
import tflite_runtime.interpreter as tflite
import shutil
import pandas as pd

In [ ]:
# captcha_symbols="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ#[]+:></%{}" # I didn't see it in mine but potentially '\'
captcha_symbols="0123456789ceghijklnpoqstuvwxyzABCDFJKMPQRSTUVWXYZ#[]+:></%{}\-|_ " 

In [ ]:
# Preprocess
def preprocess(raw_data) :
    return smallPreprocess(raw_data)

def erase_circles(img, circles):
    circles = circles[0]  # hough circles returns a nested list for some reason
    for circle in circles:
        center = (round(circle[0]),round(circle[1]))
        img = cv2.circle(img, center, radius=round(circle[2]), color=255, thickness=1)  # erase circle by making it white (to match the image background)
    return img

def detect_and_remove_circles(img):
    hough_circle_locations = cv2.HoughCircles(img, method=cv2.HOUGH_GRADIENT, dp=1, minDist=1, param1=50, param2=5, minRadius=0, maxRadius=2)
    if hough_circle_locations is not None:
        img = erase_circles(img, hough_circle_locations)
    return img

def smallPreprocess(raw_data):
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2GRAY) # Gray Image
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)  # dilate image to initial stage (erode works similar to dilate because we thresholded the image the opposite way)
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # erode just a bit to polish fine details
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Back to Colour for channel
    image = np.array(img) / 255.0
    (c, h, w) = image.shape
    image = image.reshape([-1, c, h, w])
    return image

def pyPreprocess(raw_data):
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2GRAY) # Gray Image
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)  # dilate image to initial stage (erode works similar to dilate because we thresholded the image the opposite way)
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # erode just a bit to polish fine details
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Back to Colour for channel
    # image = np.array(img) / 255.0
    image = np.array(img, dtype=np.float32) / 255.0
    (c, h, w) = image.shape
    image = image.reshape([-1, c, h, w])
    return image

def bigPreprocess(raw_data) :
    #   Back to Black
    img = cv2.cvtColor(raw_data, cv2.COLOR_BGR2GRAY)
    # run some basic tests to get rid of easy-to-remove noise -- first pass
    img = ~img  # white letters, black background
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)  # weaken circle noise and line noise
    img = ~img  # black letters, white background
    img = scipy.ndimage.median_filter(img, (5, 1))  # remove line noise
    img = scipy.ndimage.median_filter(img, (1, 3))  # weaken circle noise
    img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations=1)  # dilate image to initial stage (erode works similar to dilate because we thresholded the image the opposite way)
    img = scipy.ndimage.median_filter(img, (3, 3))  # remove any final 'weak' noise that might be present (line or circle)
    # detect any remaining circle noise
    img = detect_and_remove_circles(img)  # after dilation, if concrete circles exist, use hough transform to remove them
    # eradicate any final noise that wasn't removed previously -- second pass
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # actually performs erosion
    img = scipy.ndimage.median_filter(img, (5, 1))  # finally completely remove any extra noise that remains
    img = cv2.erode(img, np.ones((3, 3), np.uint8), iterations=2)  # dilate image to make it look like the original
    img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=1)  # erode just a bit to polish fine details
    #Edge Detection
    # img = cv2.Canny(img, 100, 200)
    # Back to Colour
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    # Format - (different for pi)
    image = np.array(img) / 255.0
    (c, h, w) = image.shape
    return image.reshape([-1, c, h, w])

In [ ]:
# Generate
#!/usr/bin/env python3
# from google.colab import drive
# drive.mount('/content/drive')
def main():
    width = 128
    height = 64
    length = 6
    #Train
    # count = 160000 #160000 - 10m 58s #16000 - 1m 6s #100 - 0s
    # output_dir = '/content/train_set' # image set
    # outputFile = '/content/train_labels.txt' #  labels
    # #Val
    # count = 16000 #160000 - 10m 58s #16000 - 1m 6s #100 - 0s
    # output_dir = '/content/val_set' # image set
    # outputFile = '/content/val_labels.txt' #  labels
    # #Test
    count = 100 #160000 - 10m 58s #16000 - 1m 6s #100 - 0s
    output_dir = '/content/test_set' # image set
    outputFile = '/content/test_labels.txt' #  labels

    captcha_symbols="%{}[]()#:><+-|_1234567890ABCDFMPQRSTUVWXYZecghjknx" 

    for x in range(0, 15):
      captcha_symbols = captcha_symbols + ' '

    captcha_generator = captcha.image.ImageCaptcha(width=width, height=height)

    # symbols_file = open(symbols, 'r')
    # captcha_symbols = symbols_file.readline().strip()
    # symbols_file.close()

    print("Generating captchas with symbol set {" + captcha_symbols + "}")

    if not os.path.exists(output_dir):
        print("Creating output directory " + output_dir)
        os.makedirs(output_dir)

    filenames = []
    index = 0
  
    for i in range(count):
        caplength = np.random.randint(1,7)
        random_str = ''.join([random.choice(captcha_symbols) for j in range(caplength)])
        img_name = 'img_'+str(i)+'.png'
        image_path = os.path.join(output_dir, img_name)
        if os.path.exists(image_path):
            version = 1
            while os.path.exists(os.path.join(output_dir, random_str + '_' + str(version) + '.png')):
                version += 1
            image_path = os.path.join(output_dir, random_str + '_' + str(version) + '.png')

        image = np.array(captcha_generator.generate_image(random_str))
        cv2.imwrite(image_path, image)
        with open(outputFile,'+a') as f:
            random_str.strip()
            f.write(img_name + "," + random_str.replace(" ", "") + "\n")

if __name__ == '__main__':
    main()

In [ ]:
# Generate
#!/usr/bin/env python3
from google.colab import drive
drive.mount('/content/drive')
def main():
    width = 128
    height = 64
    length = 6
    count = 1000 #100 #140800 #128000 - 40m
    # output_dir = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 2/test_set' #test set
    # output_dir = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 2/train_set' #training set
    outputFile = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 2/training_lables'
    output_dir = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 2/validation_set' #validation set
    # Appedning symbol set with \ and -
    captcha_symbols="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ#[]+:></%{}\-" # I didn't see it in mine but potentially '\'    # symbols = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 2/symbols.txt'
    # for x in range(0, 9):
    #   captcha_symbols = captcha_symbols + ' '

    captcha_generator = captcha.image.ImageCaptcha(width=width, height=height)

    # symbols_file = open(symbols, 'r')
    # captcha_symbols = symbols_file.readline().strip()
    # symbols_file.close()

    print("Generating captchas with symbol set {" + captcha_symbols + "}")

    if not os.path.exists(output_dir):
        print("Creating output directory " + output_dir)
        os.makedirs(output_dir)

    filenames = []
    index = 0
  
    for i in range(count):
        caplength = np.random.randint(1,7)
        random_str = ''.join([random.choice(captcha_symbols) for j in range(caplength)])
        img_name = 'img_'+str(i)+'.png'
        image_path = os.path.join(output_dir, img_name)
        if os.path.exists(image_path):
            version = 1
            while os.path.exists(os.path.join(args.output_dir, random_str + '_' + str(version) + '.png')):
                version += 1
            image_path = os.path.join(args.output_dir, random_str + '_' + str(version) + '.png')

        image = np.array(captcha_generator.generate_image(random_str))
        cv2.imwrite(image_path, image)
        with open(outputFile,'+a') as f:
            f.write(img_name + ", " + random_str + "\n")

    # for i in range(count):
    #     random_str = ''.join([random.choice(captcha_symbols) for j in range(length)])

    #     filenames.append(random_str)

    #     image_path = os.path.join(output_dir, str(index)+'.png')
        
    #     if os.path.exists(image_path):
    #         version = 1
    #         while os.path.exists(os.path.join(output_dir, str(index) + '_' + str(version) + '.png')):
    #             version += 1
    #         image_path = os.path.join(output_dir, str(index) + '_' + str(version) + '.png')

    #     image = np.array(captcha_generator.generate_image(random_str))
    #     cv2.imwrite(image_path, image)
    #     index = index + 1

    # pd.DataFrame(filenames).to_csv("/content/drive/MyDrive/Year V/Scalable Computing/Practical 2/validation_set_names.csv")

if __name__ == '__main__':
    main()

In [ ]:
# Train
#!/usr/bin/env python3

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Build a Keras model given some parameters
def create_model(captcha_length, captcha_num_symbols, input_shape, model_depth=5, module_size=2):
  input_tensor = keras.Input(input_shape)
  x = input_tensor
  for i, module_length in enumerate([module_size] * model_depth):
      for j in range(module_length):
          x = keras.layers.Conv2D(32*2**min(i, 3), kernel_size=3, padding='same', kernel_initializer='he_uniform')(x)
          x = keras.layers.BatchNormalization()(x)
          x = keras.layers.Activation('relu')(x)
      x = keras.layers.MaxPooling2D(2)(x)

  x = keras.layers.Flatten()(x)
  x = [keras.layers.Dense(captcha_num_symbols, activation='softmax', name='char_%d'%(i+1))(x) for i in range(captcha_length)]
  model = keras.Model(inputs=input_tensor, outputs=x)

  return model

# A Sequence represents a dataset for training in Keras
# In this case, we have a folder full of images
# Elements of a Sequence are *batches* of images, of some size batch_size
class ImageSequence(keras.utils.Sequence):
    def __init__(self, directory_name, label_file, batch_size, captcha_length, captcha_symbols, captcha_width, captcha_height):
        self.directory_name = directory_name
        self.batch_size = batch_size
        self.captcha_length = captcha_length
        self.captcha_symbols = captcha_symbols
        self.captcha_width = captcha_width
        self.captcha_height = captcha_height
        with open(label_file,"+r") as f:
          labelList = f.readlines()
          self.labels = dict(zip(map(lambda x:x.split(",")[0],labelList),map(lambda x:x.split(",")[1].strip(),labelList)))
        file_list = os.listdir(self.directory_name)
        self.files = dict(zip(map(lambda x: x.split('.')[0], file_list), file_list))
        self.used_files = []
        self.count = len(file_list)
        print('dir name ' + self.directory_name + '/n')
        print(self.count)
        # print(self.files['img_90160'])
        # print(self.labels)
        # random_image_label = self.labels['img_1.png'].split('_')[0]
        # print(random_image_label)
        # print(self.files['img_2'])
        # if len(random_image_label) < self.captcha_length :
        #     balance = self.captcha_length - len(random_image_label)
        #     for x in range(0, balance) :
        #       random_image_label = random_image_label + ' '
        # for j, ch in enumerate(random_image_label):
        #   print(random_image_label)
        #   print(j)

    def __len__(self):
        return int(np.floor(self.count / self.batch_size))

    def __getitem__(self, idx):
        X = np.zeros((self.batch_size, self.captcha_height, self.captcha_width, 3), dtype=np.float32)
        y = [np.zeros((self.batch_size, len(self.captcha_symbols)), dtype=np.uint8) for i in range(self.captcha_length)]

        for i in range(min(self.batch_size, len(self.files))):
            random_image_name = random.choice(list(self.labels.keys()))
            random_image_label = self.labels[random_image_name]
            random_image_file = self.files[random_image_name[:-4]]

            # We've used this image now, so we can't repeat it in this iteration
            self.used_files.append(self.files.pop(random_image_name[:-4]))
            self.labels.pop(random_image_name)

            # We have to scale the input pixel values to the range [0, 1] for
            # Keras so we divide by 255 since the image is 8-bit RGB
            raw_data = cv2.imread(os.path.join(self.directory_name, random_image_file))
            processed_data = preprocess(raw_data)
            X[i] = processed_data

            random_image_label = random_image_label.split('_')[0]
            if len(random_image_label) < self.captcha_length :
              balance = self.captcha_length - len(random_image_label)
              for x in range(0, balance) :
                random_image_label = random_image_label + ' '
            
            # print(random_image_label)
            for j, ch in enumerate(random_image_label):
                # print('i ' + str(i))
                # print('j ' + str(j))
                y[j][i, :] = 0
                y[j][i, self.captcha_symbols.find(ch)] = 1

        return X, y

def main():
    # inputs
    width = 128
    height = 64
    length = 6 
    # symbols = 'symbols.txt'
    batch_size = 32
    epochs = 2 #8
    train_dataset = '/content/train_set/'
    train_labels_file = '/content/train_labels.txt'
    validate_dataset = '/content/val_set/'
    val_labels_file = '/content/val_labels.txt'
    output_model_name = '/content/train_model_10'
    input_model = output_model_name #None

    # captcha_symbols = None
    # with open(symbols) as symbols_file:
    #     captcha_symbols = symbols_file.readline()

    with tf.device('/device:GPU:0'):

        model = create_model(length, len(captcha_symbols), (height, width, 3))

        if input_model is not None:
            model.load_weights('/content/train_model_09'+'.h5')

        model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(1e-3, amsgrad=True),
                      metrics=['accuracy'])

        model.summary()

        # test_data = ImageSequence('/content/drive/MyDrive/Practical 2/test_set/', '/content/drive/MyDrive/Practical 2/test_labels.txt', 32, 6, "avnjbafibu", 128, 64)
        training_data = ImageSequence(train_dataset, train_labels_file, batch_size, length, captcha_symbols, width, height)
        validation_data = ImageSequence(validate_dataset, val_labels_file, batch_size, length, captcha_symbols, width, height)

        callbacks = [keras.callbacks.EarlyStopping(patience=3),
                      # keras.callbacks.CSVLogger('log.csv'),
                      keras.callbacks.ModelCheckpoint(output_model_name+'.h5', save_best_only=False)]

        # Save the model architecture to JSON
        with open(output_model_name+".json", "w") as json_file:
            json_file.write(model.to_json())

        try:
            model.fit_generator(generator=training_data,
                                validation_data=validation_data,
                                epochs=epochs,
                                callbacks=callbacks,
                                use_multiprocessing=True)
        except KeyboardInterrupt:
            print('KeyboardInterrupt caught, saving current weights as ' + output_model_name+'_resume.h5')
            model.save_weights(output_model_name+'_resume.h5')

if __name__ == '__main__':
    main()

In [ ]:
#   Classify
#!/usr/bin/env python3

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def decode(characters, y):
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])

def main():
    model_name = '/content/train_model_09'
    captcha_dir = '/content/test_set/'
    # captcha_dir = '/content/drive/MyDrive/Ruth/brennar5_imgs'
    label_file = '/content/test_labels.txt'
    output = '/content/classify_train_model_05_output.csv'
    # symbols = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 1/symbols.txt'

    labels = None
    with open(label_file,"+r") as f:
      labelList = f.readlines()
      labels = dict(zip(map(lambda x:x.split(",")[0],labelList),map(lambda x:x.split(",")[1].strip(),labelList)))

    # print(labels['img_0.png'])

    # symbols_file = open(symbols, 'r')
    # captcha_symbols = symbols_file.readline().strip()
    # symbols_file.close()

    print("Classifying captchas with symbol set {" + captcha_symbols + "}")
    correct = 0
    incorrect = 0

    with tf.device('/cpu:0'):
        with open(output, 'w') as output_file:
            json_file = open(model_name+'.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = keras.models.model_from_json(loaded_model_json)
            model.load_weights(model_name+'.h5')
            # model.load_weights('/content/drive/MyDrive/Practical 2/train_model_01'+'_resume'+'.h5')
            # model.load_weights(model_name+'_resume.h5')
            model.compile(loss='categorical_crossentropy',
                          optimizer=keras.optimizers.Adam(1e-3, amsgrad=True),
                          metrics=['accuracy'])
            


            for x in os.listdir(captcha_dir):
                # load image and preprocess it
                raw_data = cv2.imread(os.path.join(captcha_dir, x))

                #   Preprocess Image
                image = preprocess(raw_data)

                prediction = model.predict(image)
                predictedAnswer = decode(captcha_symbols, prediction)
                output_file.write(x + ", " + predictedAnswer + "\n")
                predictedAnswer.strip()
                predictedAnswer = predictedAnswer.replace(" ", "")
                # predictedAnswer = predictedAnswer.replace("_", "")

                # answer = x[:-4]
                answer = labels.get(x)
                print('Classified ' + x + ' ' + answer + '///' + predictedAnswer)
                if (answer == predictedAnswer) :
                    correct = correct + 1
                    print('^ CORRECT ^')
                else :
                    incorrect = incorrect + 1

    print('correct = '+str(correct))
    print('incorrect ='+str(incorrect))


if __name__ == '__main__':
    main()

In [ ]:
#!/usr/bin/env python3
# Converter

import tensorflow as tf
import tensorflow.keras as keras

def main(model_name):
    with open(f'/content/{model_name}.json') as f:
        model = f.read()
    model = keras.models.model_from_json(model)
    model.load_weights(f'/content/{model_name}.h5')
    cvt = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite = cvt.convert()
    with open(f'/content/converted_{model_name}.tflite', 'wb') as f:
        f.write(tflite)

if __name__ == '__main__':
    model_name = 'train_model_09'
    main(model_name)

In [ ]:
# Classify - Lite
#!/usr/bin/env python3
import os
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def decode(characters, y):
    y = np.argmax(np.array(y), axis=1)
    #y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])

def main():
    
    model_name = '/content/converted_train_model_09.tflite'
    # captcha_dir = '/content/test_set'
    captcha_dir = '/content/drive/MyDrive/brennar5_imgs'
    # output = '/content/converted_5_output.csv'
    output = '/content/brennar5.csv'
    # symbols = '/content/drive/MyDrive/Year V/Scalable Computing/Practical 1/symbols.txt'

    # label_file = '/content/test_labels.txt'
    # labels = None
    # with open(label_file,"+r") as f:
    #   labelList = f.readlines()
    #   labels = dict(zip(map(lambda x:x.split(",")[0],labelList),map(lambda x:x.split(",")[1].strip(),labelList)))


# Pi Addresses
    # model_name = '/users/ugrad/brennar5/captcha_detection/converted_test.tflite'
    # captcha_dir = '/users/ugrad/brennar5/captcha_detection/testers_4'
    # output = '/users/ugrad/brennar5/captcha_detection/converted_4_output.csv'

    # symbols_file = open(symbols, 'r')
    # captcha_symbols = symbols_file.readline().strip()
    # symbols_file.close()

    # captcha_symbols = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

    print("Classifying captchas with symbol set {" + captcha_symbols + "}")
    correct = 0
    incorrect = 0

    with open(output, 'w') as output_file:
        
        tf_interpreter = tflite.Interpreter(model_name)
        tf_interpreter.allocate_tensors()
        input_tf = tf_interpreter.get_input_details()
        output_tf = tf_interpreter.get_output_details()
        # print(input_tf)
        # print(output_tf)

        files = os.listdir(captcha_dir)
        files = sorted(files)

        for x in files:
            # Load & Preprocess Image - Currently to work with test
            raw_data = cv2.imread(os.path.join(captcha_dir, x))
            image = pyPreprocess(raw_data)
            
            tf_interpreter.set_tensor(input_tf[0]['index'],image)
            tf_interpreter.invoke()
            prediction = []
            for output_node in output_tf:
                prediction.append(tf_interpreter.get_tensor(output_node['index']))
            prediction = np.reshape(prediction,(len(output_tf),-1))
            predictedAnswer = decode(captcha_symbols, prediction)
            predictedAnswer.strip()
            predictedAnswer = predictedAnswer.replace(" ", "")
            # predictedAnswer = predictedAnswer.replace("_", "")
            output_file.write(x + "," + predictedAnswer + "\n")
            
            # answer = labels.get(x)
            # print('Classified ' + answer + '///' + predictedAnswer)

            answer = x[:-4]
            # answer = labels.get(x)
            if (answer == predictedAnswer) :
                correct = correct + 1
                print('^ CORRECT ^')
            else :
                incorrect = incorrect + 1

    print('correct = '+str(correct))
    print('incorrect ='+str(incorrect))
                
if __name__ == '__main__':
    main()